In [2]:
import config
import cx_Oracle
import re
import pandas as pd
import csv
import pprint

pattern = re.compile(r'\s+')

In [13]:
# connect to Oracle DB
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding = "UTF-8"
    )
    print("Connected")
    print(connection.version)
except cx_Oracle.Error as error:
    print(error)

cursor = connection.cursor()

Connected
12.2.0.1.0


In [43]:
# set up style view and get the geometrytype of that style's feature
view_name = "V_LV_SEC_LN_1"
feature_name = view_name[2:]
print(feature_name)
sql = "Select g3e_geometrytype from G3E_COMPONENT where G3E_NAME = '{name}'".format(name = feature_name)
cursor.execute(sql)
geometrytype = cursor.fetchall()[0][0]
print(geometrytype)

LV_SEC_LN_1
PolylineGeometry


In [44]:
# hardcode condition to know which styletable for the given style view
if geometrytype == "PolygonGeometry":
    styletable = 'G3E_AREASTYLE'
elif geometrytype == "PolylineGeometry":
    styletable = 'G3E_COMPOSITELINESTYLE'
elif geometrytype == "OrientedPointGeometry":
    styletable = 'G3E_POINTSTYLE'
elif geometrytype == "TextPointGeometry":
    styletable = 'G3E_TEXTSTYLE'
elif geometrytype == "CompositePolylineGeometry":
    styletable = 'G3E_COMPOSITELINESTYLE'

print(styletable)

G3E_COMPOSITELINESTYLE


In [45]:
# extract text that create the given style view
sql = "SELECT TEXT FROM USER_VIEWS WHERE VIEW_NAME = '{view_name}'".format(view_name = view_name)
cursor.execute(sql)
fetch = cursor.fetchall()
txts = fetch[0][0]
txts = txts.rpartition("CASE")[2]
txts = txts.rpartition("ELSE")[0]
txts = re.sub(pattern,'',txts)
txts = txts.split("WHEN")

In [46]:
# extract cause(logic condition) and effect(styleid) into dict
logics = {}

spl_chr = "THEN"
for txt in txts:
    if txt != "":
        styleid = txt.rpartition(spl_chr)[2]
        index = len(styleid)
        for i in styleid:
            if not i.isdigit():
                index = styleid.index(i)
                break
        styleid = styleid[:index]
        logics.update({styleid : txt.rpartition(spl_chr)[0]})
    
print(logics)

{'15750010': "TRIM(E.CODETEXT)='UNKNOWNROUTE'ANDTRIM(D.CODETEXT)='INS'", '15750020': "TRIM(E.CODETEXT)='UNKNOWNROUTE'ANDTRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')", '15750030': "TRIM(E.CODETEXT)='UNKNOWNROUTE'ANDTRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')", '15750040': "TRIM(E.CODETEXT)='UNKNOWNROUTE'ANDTRIM(D.CODETEXT)='APA'", '15750050': "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='SUBMARINECABLE'ANDTRIM(C.WATER_TREE)='YES'", '15750060': "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='SUBMARINECABLE'ANDTRIM(C.INGRESS_STATUS)='YES'", '15750070': "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='SUBMARINECABLE'", '15750080': "TRIM(D.CODETEXT)='INS'ANDTRIM(C.WATER_TREE)='YES'", '15750090': "TRIM(D.CODETEXT)='INS'ANDTRIM(C.INGRESS_STATUS)='YES'", '15750100': "TRIM(D.CODETEXT)='INS'", '15750110': "TRIM(D.CODETEXT)IN('ABN','RES')ANDTRIM(E.CODETEXT)='SUBMARINECABLE'ANDTRIM(C.WATER_TREE)='YES'", '15750120': "TRIM(D.CODETEXT)IN('ABN','RES')ANDTRIM(E.CODETEXT)='SUBMARINECABLE'ANDTRIM(C.INGRESS_STATUS)='

In [47]:
# get the relevant information for the given style id
styles = {}
# check if compositeline, as they have a seperate method of getting style info
if styletable != 'G3E_COMPOSITELINESTYLE':
    for styleid in logics.keys():
        sqlstyle = "SELECT * FROM {table} WHERE g3e_sno = '{styleid}'".format(table = styletable, styleid = styleid)
        df=pd.read_sql(sqlstyle,con=connection)
        data = df.to_dict(orient = 'list')
        # remove useless stuff
        data.pop('G3E_SNO', None)
        data.pop('G3E_USERNAME', None)
        data.pop('G3E_EDITDATE', None)

        styles.update({styleid:data})
else:
    for styleid in logics.keys():
        sqlstyle = "SELECT b.* FROM G3E_COMPOSITELINESTYLE a JOIN g3e_linestyle b on a.g3e_line1 = b.g3e_sno WHERE a.g3e_sno = '{styleid}'".format(styleid = styleid)
        df1=pd.read_sql(sqlstyle,con=connection)
        # remove useless stuff
        df1 = df1.drop(df1.columns[[0,1,3,4,5,6,9,10,11]], axis = 1)

        sqlstyle = "SELECT c.* FROM G3E_COMPOSITELINESTYLE a JOIN g3e_linestyle c on a.g3e_line2 = c.g3e_sno WHERE a.g3e_sno = '{styleid}'".format(styleid = styleid)
        df2=pd.read_sql(sqlstyle,con=connection)
        # remove useless stuff
        df2 = df2.drop(df2.columns[[0,1,3,4,5,6,9,10,11]], axis = 1)

        df = pd.concat([df1,df2], axis = 1)
        # rename column that has the same name
        cols = pd.Series(df.columns)            
        for dup in df.columns[df.columns.duplicated(keep=False)]:
            cols[df.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) if d_idx != 0 else dup for d_idx in range(df.columns.get_loc(dup).sum())])
        df.columns = cols
        data = df.to_dict(orient = 'list')

        styles.update({styleid:data})

#print(styles)

In [48]:
printdicts = {}
for styleid in logics.keys():
    printdicts.update({styleid: styles[styleid]})
    printdicts[styleid].update({"logic": logics[styleid]})

df = pd.DataFrame.from_dict({(i): printdicts[i]
                                for i in printdicts.keys()},
                                orient = 'index')

In [49]:
filename = view_name + ".csv"
csvdata = df.to_csv()
with open(filename,'w') as f:
    f.write(csvdata)
    

In [11]:
# close connection
if connection:
    cursor.close()
    connection.close()
    print("Connection closed")

Connection closed
